# Load dataset

In [20]:
from glob import glob
import os
import json 

path = '../prompting/outputs/mistral7b_c1_hf_partial/'
dialog_files = glob(os.path.join(path, '*.json'))

data = []
for p in dialog_files:
    with open(p, 'r') as f:
        data.append(json.loads(f.read()))
        

print(len(data))

25


In [21]:
lens = [len(x['dialog']) for x in data]

print(min(lens))

min_dialog_turns = min(lens)
    

5


# Process conversations and add an id to them

In [29]:
from uuid import uuid4

conv_ids = {}

for ex in data:
    conv_str = '_'.join(ex['dialog'][:min_dialog_turns])
    if conv_str not in conv_ids:
        conv_ids[conv_str] = str(uuid4())
        
    ex['id'] = conv_ids[conv_str]

In [30]:
print("total conversations:", len(conv_ids))

total conversations: 25


# split across conversations for training and test

In [31]:
import numpy as np

all_ids = list(conv_ids.values())
np.random.shuffle(all_ids)

val_size = 5
test_size = 5

test_ids = all_ids[:test_size]
val_ids = all_ids[test_size:test_size+val_size]
train_ids = all_ids[test_size+val_size:]

train = [d for d in data if d['id'] in train_ids]
val = [d for d in data if d['id'] in val_ids]
test = [d for d in data if d['id'] in test_ids]

print("training size: ", len(train))
print("validation size: ", len(val))
print("test size: ", len(test))

training size:  15
validation size:  5
test size:  5


In [32]:
import json

def write_jsonl(path, data):
    with open(path, 'w') as f:
        for ex in data:
            f.write(json.dumps(ex) + "\n")
            
            
write_jsonl('data/train.json', train)
write_jsonl('data/val.json', val)
write_jsonl('data/test.json', test)